In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
n_collect = 100
coverage0 = 0.2

In [3]:
np.linspace(0, 0.8, 33)

array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  ])

In [4]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 0.8, 33), tp="VulACF"):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)
            
            idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Plain') or c.startswith('ACF_Vul')]

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [5]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=n_collect, tp='VulACF')
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=n_collect, tp='PlainACF')
    mss = pd.concat([mss0, mss1])
    mss_stats = pd.concat([mss_stats0, mss_stats1])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_lo
dy_hi


In [6]:
def collect_fu(pars_post, n_collect=20, coverage=coverage0, tp="VulACF"):
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'VulACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 2}},
        'Vul_2_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 2}},
        'Vul_3_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 3}},
        'Vul_3_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 3}},
    }
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
            
            idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Vulfu') or c.startswith('ACF_Vul')]

            ms = ms[idx].reset_index().assign(Key = i, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [7]:
for folder in ['dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fu(pars_post, n_collect=n_collect, coverage=0.2)
    mss_stats = mss_stats.fillna(0)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_hi


In [8]:
mss_stats

,Key,Scenario,Pop0,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,ACF_Vul_Sym,ACF_Vul_Vul,...,ACF_VulFu_Sym,ACF_VulFu_Vul,ACF_VulFu_CXR,ACF_VulFu_Xpert,ACF_VulFu_Yield,ACF_VulFu_TP,ACF_VulFu_DS_Fl,ACF_VulFu_DR_Fl,ACF_VulFu_DR_Sl,Coverage
0,23,Baseline,2.383208,0.087795,0.010880,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
1,23,Vul_0_0,2.383208,0.081129,0.009623,4.412863,0.917456,0.246010,4.407961,4.407961,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
2,23,Vul_2_3,2.383208,0.078730,0.009225,4.412861,0.917346,0.245671,4.407961,4.407961,...,4.407961,0.0,0.0,0.127660,0.003876,0.002248,0.002153,0.0,0.000095,0.2
3,23,Vul_2_6,2.383208,0.079309,0.009325,4.412845,0.917383,0.245759,4.407961,4.407961,...,4.407961,0.0,0.0,0.064222,0.002297,0.001483,0.001420,0.0,0.000063,0.2
4,23,Vul_3_3,2.383208,0.078289,0.009148,4.412867,0.917324,0.245604,4.407961,4.407961,...,4.407961,0.0,0.0,0.156475,0.004884,0.002891,0.002770,0.0,0.000122,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,1490,Vul_0_0,2.383804,0.068685,0.007816,4.413431,0.918624,0.232073,4.409063,4.409063,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
596,1490,Vul_2_3,2.383804,0.067627,0.007620,4.413550,0.918472,0.231875,4.409063,4.409063,...,4.409063,0.0,0.0,0.128279,0.004945,0.001484,0.001427,0.0,0.000057,0.2
597,1490,Vul_2_6,2.383804,0.067943,0.007679,4.413487,0.918538,0.231940,4.409063,4.409063,...,4.409063,0.0,0.0,0.064436,0.002717,0.000985,0.000947,0.0,0.000038,0.2
598,1490,Vul_3_3,2.383804,0.067376,0.007571,4.413578,0.918437,0.231826,4.409063,4.409063,...,4.409063,0.0,0.0,0.157221,0.006148,0.001909,0.001835,0.0,0.000074,0.2


In [9]:
def collect_fux(pars_post, n_collect=20, coverage=coverage0, tp="VulACF"):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()

    # Test run
    pars = pars_post[0]
    y1, pars = m.find_baseline(pars, 2022)
    _, ms, _ = m.simulate_onward(y1, pars, intv={'VulACF': {'Coverage': 0.01, 'FollowUp': 1, 'Duration': 1}}, t_end=2025)

    idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Vul')]
    print(idx)
    
    #
    
    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
    
        _, ms, _ = m.simulate_onward(y1, pars, intv={}, t_end=2031)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        ms_stats = {'Key': i, 'Scenario': 'Baseline', 'Pop0': ms.Pop.iloc[0], 
                    'FollowUp': 0, 'Duration': 0}
        for key in idx[1:]:
            try:
                ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
            except KeyError:
                ms_stats[key] = 0

        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            if n_fu > 0:
                fu = 1 / n_fu
            else:
                fu = 0
            for dur in np.linspace(0, 4, 9):
                intv = {'VulACF': {'Coverage': coverage, 'FollowUp': fu, 'Duration': dur}}
                
                _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
                try:
                    ms = ms.reset_index().assign(Key = i, FollowUp=fu, Duration=dur)
                    mss.append(ms)

                    ms_stats = {'Key': i, 'Scenario': 'Intervention', 'Pop0': ms.Pop.iloc[0], 
                                'FollowUp': fu, 'Duration': dur}

                    for key in idx[1:]:
                        try:
                            ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
                        except KeyError:
                            ms_stats[key] = 0

                    mss_stats.append(ms_stats)
                except TypeError:
                    pass
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [10]:
for folder in ['dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fux(pars_post, n_collect=n_collect, coverage=0.2)
    mss_stats = mss_stats.fillna(0)
    mss.to_csv(f'{out_path}/Sim_VulACF_fudur.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_fudur_stats.csv')

dy_hi
['Pop', 'IncR', 'MorR', 'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Sym', 'ACF_Vul_Vul', 'ACF_Vul_CXR', 'ACF_Vul_Xpert', 'ACF_Vul_Yield', 'ACF_Vul_TP', 'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 'ACF_VulFu_Footfall', 'ACF_VulFu_Screened', 'ACF_VulFu_Confirmed', 'ACF_VulFu_Sym', 'ACF_VulFu_Vul', 'ACF_VulFu_CXR', 'ACF_VulFu_Xpert', 'ACF_VulFu_Yield', 'ACF_VulFu_TP', 'ACF_VulFu_DS_Fl', 'ACF_VulFu_DR_Fl', 'ACF_VulFu_DR_Sl']


In [11]:
mss_stats

,Key,Scenario,Pop0,FollowUp,Duration,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,...,ACF_VulFu_Sym,ACF_VulFu_Vul,ACF_VulFu_CXR,ACF_VulFu_Xpert,ACF_VulFu_Yield,ACF_VulFu_TP,ACF_VulFu_DS_Fl,ACF_VulFu_DR_Fl,ACF_VulFu_DR_Sl,Coverage
0,15,Baseline,2.383522,0.000000,0.0,0.094133,0.012705,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
1,15,Intervention,2.383522,0.000000,0.0,0.086088,0.011103,4.413894,0.917901,0.227688,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
2,15,Intervention,2.383522,0.000000,0.5,0.086079,0.011101,4.413891,0.917901,0.227687,...,4.408541,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
3,15,Intervention,2.383522,0.000000,1.0,0.086094,0.011102,4.413892,0.917901,0.227688,...,4.408541,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
4,15,Intervention,2.383522,0.000000,1.5,0.086095,0.011102,4.413892,0.917901,0.227688,...,4.408541,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,1496,Intervention,2.383461,0.166667,2.0,0.086089,0.011129,4.413913,0.918411,0.216117,...,4.408428,0.0,0.0,0.187045,0.008849,0.002126,0.002047,0.0,0.000078,0.2
6396,1496,Intervention,2.383461,0.166667,2.5,0.085925,0.011091,4.413948,0.918376,0.216077,...,4.408428,0.0,0.0,0.210235,0.010024,0.002471,0.002380,0.0,0.000091,0.2
6397,1496,Intervention,2.383461,0.166667,3.0,0.085795,0.011060,4.413977,0.918347,0.216045,...,4.408428,0.0,0.0,0.228911,0.010967,0.002745,0.002644,0.0,0.000101,0.2
6398,1496,Intervention,2.383461,0.166667,3.5,0.085690,0.011035,4.414001,0.918323,0.216018,...,4.408428,0.0,0.0,0.244233,0.011736,0.002965,0.002856,0.0,0.000109,0.2
